# Deep Learning assignment #1

***Student:*** [Isac Pasianotto](https://github.com/IsacPasianotto/dlhw)

In [27]:
import torch
from typing import Iterable, List, Tuple
from torch import Tensor
from itertools import product

## 0. Dataset definition

The considered dataset is the set of all possible 6-bit long binary sequences. Since the task the network is supposed to perform is to check if a given sequence is palindromic, the dataset is labeled accordingly.

In [2]:
def is_symmetric(iterable: Iterable) -> float:
    """
    Check if an iterable is palindromic.
    :param iterable: an iterable object to check (in our case, a torch tensor)
    :return: 1 if the iterable is palindromic, 0 otherwise
    """
    return 1. if all(a == b for a, b in zip(iterable, reversed(iterable))) else 0.

# Ensure that the function works as expected
assert is_symmetric( torch.tensor([0, 1, 0, 1, 1, 0]) ) == 0.
assert is_symmetric( torch.tensor([0, 1, 0, 0, 1, 0]) ) == 1.

In [40]:
def build_dataset(nbits: int = 6, dtype: torch.dtype = torch.float16, unbalanced: bool = True, seed: int = 42 ) -> Tuple[Tensor, Tensor]:
    """
    Build the dataset of all possible binary sequences of length nbits and their corresponding labels.
    :param nbits: the length (integer) of the binary sequences. Default is 6.
    :param dtype: the data type of the tensors. Default is torch.float16.
    :param unbalanced: if True, the dataset will contain all the possible binary sequences of length nbits. If False, the palindromic sequences will be repeated more time to balance the dataset.
    :param seed: the seed for the random number generator. Default is 42, which is the answer to the Ultimate Question of Life, the Universe, and Everything.
    :return: Two tensors: the first one contains all the binary sequences, the second one contains the corresponding labels.
    """
    bin_seq: List[Tuple[int]] = list(product([0, 1], repeat=nbits))  # generate all possible binary sequences of length nbits
    sim_true: List[Tuple[int]] = [seq for seq in bin_seq if is_symmetric(seq)]
    sim_false: List[Tuple[int]] = [seq for seq in bin_seq if seq not in sim_true]

    sim_true: Tensor = torch.tensor(sim_true, dtype=dtype)
    sim_false: Tensor = torch.tensor(sim_false, dtype=dtype)

    if unbalanced:
        x_data: Tensor = torch.cat([sim_true, sim_false], dim=0)
        y_data: Tensor = torch.cat([torch.ones(sim_true.shape[0], dtype=dtype), torch.zeros(sim_false.shape[0], dtype=dtype)], dim=0)
    else:
        balance_ratio: int = sim_false.shape[0] // sim_true.shape[0]
        x_data: Tensor = torch.cat([sim_false, torch.cat([sim_true]*balance_ratio, dim=0)], dim=0)
        y_data: Tensor = torch.cat([torch.zeros(sim_false.shape[0], dtype=dtype), torch.ones(sim_true.shape[0]*balance_ratio, dtype=dtype)], dim=0)
        
    n_seq: int = x_data.shape[0]
    rndidx: Tensor[int] = torch.randperm(n_seq, generator=torch.Generator().manual_seed(seed))
    # Shuffle the dataset
    x_data = x_data[rndidx]
    y_data = y_data[rndidx]
    
    return x_data, y_data

x_data, y_data = build_dataset()
x_data_balanced, y_data_balanced = build_dataset(unbalanced=False)

## 1. Network definition

The considered paper didn't provide all the needed information to reproduce exactly the experiment. From that I have evinced that:
- The input layer has 6 neurons, one for each bit of the input sequence.
- There is just 1 hidden layer with 2 neurons.
- The output layer has 1 neuron, which is the output of the network.

Hence, we can re-draw the *Fig. 1* of the paper with a more conventional representation as:  

<center><img src="./images/img-00.png" style="width: 35%;"/></center>

Moreover, there is specified that: 
- The initial weights are randomly drawn from a uniform distribution in the range [-0.3, 0.3].
- The gradient descent algorithm was not stochastic, all the dataset was used to compute the gradient.
- The train lasted for 1,425 epochs.


What is not specified is: 

- The activation function used in the hidden layer. The only mentioned non-linear function in the paper is $y_i = \frac{1}{1+e^{-x_i}}$ which is the sigmoid function and is already implemented in [PyTorch](https://pytorch.org/docs/stable/generated/https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.htmll).
- The loss function used. Since the task is a binary classification, the most common loss function is the [Binary Cross Entropy](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html).